# Projeto Grupo BT-G3


## **Integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira

<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [2]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Mounted at /content/drive


Aqui nós fazemos as importações para tratamento dos dados, pré-processamento dos dados e modelamento do Bag of Words. 

## pips

In [3]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 77.9 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [4]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

from sklearn.feature_extraction.text import CountVectorizer
import ast
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential 
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences

import nltk
import spacy
import gensim
import pickle
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

O desenvolvimento da rede neural seguiu o tutorial abaixo:
https://medium.com/@jvsavietto6/classificando-texto-com-redes-neurais-artificiais-150ef448b13d

# 2. Entendimento e Tratamento dos Dados

Rodando o dataset, para analisar seu conteúdo:

In [9]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada2')
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


In [10]:
df.columns

Index(['autor', 'sentimento', 'texto_tratado'], dtype='object')

In [11]:
df['texto_tratado']

0       ['alvarez', 'marsal', 'estar', 'conosco', 'spo...
1       ['btgpactual', 'with', 'make_repost', 'entenda...
2                             ['minuto', 'touro', 'ouro']
3           ['querer', 'saber', 'banking', 'administrar']
4       ['início', 'guerra', 'rússia', 'ucrânia', 'vel...
                              ...                        
8035                          ['excelente', 'explicação']
8036              ['atendar', 'telefone', 'amor', 'deus']
8037    ['grande', 'fiis', 'mercado', 'não', 'selecion...
8038    ['erro', 'financeiro', 'eliminar', 'parar', 'o...
8039    ['morning', 'call', 'não', 'aparecer', 'spotif...
Name: texto_tratado, Length: 8040, dtype: object

In [12]:
# Supondo que seu DataFrame seja chamado de df e a coluna seja 'texto_tratado'
df['texto_tratado'] = df['texto_tratado'].str.replace("'", "")
df['texto_tratado']

0       [alvarez, marsal, estar, conosco, sportainmet,...
1       [btgpactual, with, make_repost, entendar, impa...
2                                   [minuto, touro, ouro]
3                   [querer, saber, banking, administrar]
4       [início, guerra, rússia, ucrânia, velho, apare...
                              ...                        
8035                              [excelente, explicação]
8036                      [atendar, telefone, amor, deus]
8037    [grande, fiis, mercado, não, selecionar, princ...
8038    [erro, financeiro, eliminar, parar, ostentar, ...
8039    [morning, call, não, aparecer, spotify, atuali...
Name: texto_tratado, Length: 8040, dtype: object

# 10. Word2Vec com CBOW

## Estruturação

In [13]:
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,..."
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa..."
2,lmviapiana,2,"[minuto, touro, ouro]"
3,ricktolledo,2,"[querer, saber, banking, administrar]"
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare..."
...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]"
8036,eduardocolares,2,"[atendar, telefone, amor, deus]"
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ..."
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ..."


In [14]:
cbow = '/content/drive/MyDrive/Módulo 6/Semana 5/cbow_s50/cbow_s50.txt'

In [15]:
model_cbow = KeyedVectors.load_word2vec_format(cbow)

## Teste isolado

In [16]:
# Testando o word2vec
wordvec_test = model_cbow['projeto']

wordvec_test

array([-0.074174, -0.152088,  0.086627, -0.224567,  0.362562,  0.130683,
       -0.089179, -0.086973,  0.309501,  0.004112, -0.308202,  0.351789,
       -0.477863,  0.050276,  0.213283,  0.159895, -0.285545, -0.08832 ,
       -0.015449,  0.014816, -0.613861,  0.502556,  0.021688,  0.369492,
        0.280691,  0.016868,  0.105584, -0.180754, -0.078456,  0.148032,
        0.36293 , -0.011634,  0.412191, -0.009049,  0.010404,  0.131242,
       -0.032483, -0.133067, -0.063802,  0.434015, -0.214768, -0.072132,
        0.045601, -0.368866,  0.502808,  0.048293, -0.254894,  0.142581,
       -0.075066,  0.015646], dtype=float32)

## Definição de função

In [17]:
def create_sentence_vector(model, df):
    sentence_table = []
    for sentence in df['texto_tratado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    df["sentimentoNumerico"] = df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

## Teste de funções

In [18]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


In [19]:
#df_vec.to_csv('Word2Vec_Cbow_modelo_treinado',encoding='utf-8', index=False, header=True)

# 11. Naive Bayes + Word2Vec com CBOW

In [20]:
label = preprocessing.LabelEncoder()

In [21]:
label.fit(df_vec['sentimento'])
df_vec['sentimento'] = label.transform(df_vec['sentimento'])

In [22]:
df_vec = df_vec.dropna()
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


## Separando Treino e Teste

In [23]:
target = df_vec['sentimento']

In [24]:
feature = df_vec.iloc[:,1:50]

In [25]:
feature

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor40,Vetor41,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49
0,0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,0.168187,...,0.076954,-0.281536,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065
1,0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,0.163238,...,0.086821,-0.305640,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594
2,0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,0.202198,...,-0.008447,-0.193025,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189
3,0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,0.144342,...,0.060960,-0.254610,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563
4,0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,0.156307,...,0.067551,-0.269136,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,0.187608,...,0.014565,-0.321192,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448
8036,0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,0.150081,...,0.071114,-0.194663,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053
8037,0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,0.167985,...,0.077558,-0.300234,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422
8038,0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,0.150971,...,0.060317,-0.270532,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326


In [26]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [27]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.73      0.37       360
           1       0.51      0.30      0.38       597
           2       0.32      0.10      0.15       651

    accuracy                           0.31      1608
   macro avg       0.36      0.38      0.30      1608
weighted avg       0.38      0.31      0.28      1608



In [28]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 31.47%


# 12. Word2Vec com embedding layer

## Definição de função

In [29]:
from gensim.models import Word2Vec

# Função que treina o modelo Word2Vec no corpus do dataframe
def train_word2vec(df, column_name):
    # Obtém as frases tokenizadas
    sentences = df[column_name].tolist()
    
    # Treina o modelo Word2Vec
    model = Word2Vec(sentences, min_count=1)
    
    return model

In [30]:
# Função que define os vetores para cada palavra do vocabulario
def get_word_vectors(model, sentence):
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word]) # Append na lista de vetores
    if vectors:
        return np.sum(vectors, axis=0)/len(sentence) # Soma dos vetores para cada frase
    else:
        return np.zeros(model.vector_size)

# Criação do dataframe de vetores para cada frase
def create_word2vec_dataframe(df, column_name, model):
    sentences = df[column_name].tolist()
    vectors = [get_word_vectors(model, sentence) for sentence in sentences] # Itera para cada frase um vetor
    # Criação do dataframe
    df_vectors = pd.DataFrame(vectors, columns=[f"Vetor{i}" for i in range(model.vector_size)])
    df_word2vec = pd.concat([df, df_vectors], axis=1)
    return df_word2vec

## Teste de funções

In [31]:
model = train_word2vec(df, 'texto_tratado')

In [32]:
df_word2vec = create_word2vec_dataframe(df,'texto_tratado', model)
df_word2vec

,autor,sentimento,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.612265,-0.121134,-0.178801,-0.174028,-0.421510,0.018179,...,-0.281889,0.257685,-0.143325,0.306498,-0.234182,0.207980,-0.358231,-0.415163,0.132881,-0.113191
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa...",1,0.597896,-0.123061,-0.150154,-0.123245,-0.414788,0.018516,...,-0.258885,0.233310,-0.167665,0.288795,-0.243819,0.216081,-0.357969,-0.386116,0.109595,-0.092992
2,lmviapiana,2,"[minuto, touro, ouro]",2,0.584169,-0.033485,-0.230218,-0.246397,-0.529980,-0.002390,...,-0.224230,0.332945,-0.271342,0.243140,-0.256287,0.201317,-0.337849,-0.406413,0.191734,-0.127245
3,ricktolledo,2,"[querer, saber, banking, administrar]",2,0.547829,-0.098986,-0.120857,-0.114265,-0.307566,0.005273,...,-0.289230,0.198894,-0.125023,0.229357,-0.219359,0.255928,-0.285208,-0.372005,0.117037,-0.107042
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.614611,-0.088834,-0.184939,-0.160994,-0.445464,0.023897,...,-0.239826,0.243544,-0.161424,0.310894,-0.282802,0.195856,-0.365394,-0.403700,0.142544,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]",2,0.526041,-0.066375,-0.084716,-0.112691,-0.361852,-0.064298,...,-0.062480,0.128866,-0.120784,0.275032,-0.249917,0.083894,-0.391865,-0.397954,0.126516,-0.116072
8036,eduardocolares,2,"[atendar, telefone, amor, deus]",2,0.622338,-0.051714,-0.179536,-0.122031,-0.425095,0.035356,...,-0.244489,0.247574,-0.175288,0.318270,-0.310399,0.197917,-0.406193,-0.360116,0.113720,-0.107785
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ...",2,0.627870,-0.100649,-0.166018,-0.173724,-0.449551,0.030059,...,-0.212683,0.242995,-0.158024,0.315496,-0.302949,0.177299,-0.401135,-0.378033,0.140257,-0.061338
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.637870,-0.088951,-0.163301,-0.163432,-0.460501,0.034412,...,-0.256024,0.240638,-0.175709,0.331186,-0.290355,0.194828,-0.397644,-0.399644,0.133357,-0.081277


In [33]:
df_word2vec = df_word2vec.drop(columns=['autor', 'sentimento'])
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.612265,-0.121134,-0.178801,-0.174028,-0.421510,0.018179,-0.478128,-0.636315,...,-0.281889,0.257685,-0.143325,0.306498,-0.234182,0.207980,-0.358231,-0.415163,0.132881,-0.113191
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.597896,-0.123061,-0.150154,-0.123245,-0.414788,0.018516,-0.477988,-0.622438,...,-0.258885,0.233310,-0.167665,0.288795,-0.243819,0.216081,-0.357969,-0.386116,0.109595,-0.092992
2,"[minuto, touro, ouro]",2,0.584169,-0.033485,-0.230218,-0.246397,-0.529980,-0.002390,-0.526843,-0.695480,...,-0.224230,0.332945,-0.271342,0.243140,-0.256287,0.201317,-0.337849,-0.406413,0.191734,-0.127245
3,"[querer, saber, banking, administrar]",2,0.547829,-0.098986,-0.120857,-0.114265,-0.307566,0.005273,-0.412368,-0.573851,...,-0.289230,0.198894,-0.125023,0.229357,-0.219359,0.255928,-0.285208,-0.372005,0.117037,-0.107042
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.614611,-0.088834,-0.184939,-0.160994,-0.445464,0.023897,-0.465764,-0.662036,...,-0.239826,0.243544,-0.161424,0.310894,-0.282802,0.195856,-0.365394,-0.403700,0.142544,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.526041,-0.066375,-0.084716,-0.112691,-0.361852,-0.064298,-0.355274,-0.652298,...,-0.062480,0.128866,-0.120784,0.275032,-0.249917,0.083894,-0.391865,-0.397954,0.126516,-0.116072
8036,"[atendar, telefone, amor, deus]",2,0.622338,-0.051714,-0.179536,-0.122031,-0.425095,0.035356,-0.494136,-0.680507,...,-0.244489,0.247574,-0.175288,0.318270,-0.310399,0.197917,-0.406193,-0.360116,0.113720,-0.107785
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.627870,-0.100649,-0.166018,-0.173724,-0.449551,0.030059,-0.464931,-0.669263,...,-0.212683,0.242995,-0.158024,0.315496,-0.302949,0.177299,-0.401135,-0.378033,0.140257,-0.061338
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.637870,-0.088951,-0.163301,-0.163432,-0.460501,0.034412,-0.479991,-0.662051,...,-0.256024,0.240638,-0.175709,0.331186,-0.290355,0.194828,-0.397644,-0.399644,0.133357,-0.081277


# 13. Naive Bayes + Word2Vec com embedding layer

In [34]:
df_word2vec = df_word2vec.dropna()
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.612265,-0.121134,-0.178801,-0.174028,-0.421510,0.018179,-0.478128,-0.636315,...,-0.281889,0.257685,-0.143325,0.306498,-0.234182,0.207980,-0.358231,-0.415163,0.132881,-0.113191
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.597896,-0.123061,-0.150154,-0.123245,-0.414788,0.018516,-0.477988,-0.622438,...,-0.258885,0.233310,-0.167665,0.288795,-0.243819,0.216081,-0.357969,-0.386116,0.109595,-0.092992
2,"[minuto, touro, ouro]",2,0.584169,-0.033485,-0.230218,-0.246397,-0.529980,-0.002390,-0.526843,-0.695480,...,-0.224230,0.332945,-0.271342,0.243140,-0.256287,0.201317,-0.337849,-0.406413,0.191734,-0.127245
3,"[querer, saber, banking, administrar]",2,0.547829,-0.098986,-0.120857,-0.114265,-0.307566,0.005273,-0.412368,-0.573851,...,-0.289230,0.198894,-0.125023,0.229357,-0.219359,0.255928,-0.285208,-0.372005,0.117037,-0.107042
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.614611,-0.088834,-0.184939,-0.160994,-0.445464,0.023897,-0.465764,-0.662036,...,-0.239826,0.243544,-0.161424,0.310894,-0.282802,0.195856,-0.365394,-0.403700,0.142544,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.526041,-0.066375,-0.084716,-0.112691,-0.361852,-0.064298,-0.355274,-0.652298,...,-0.062480,0.128866,-0.120784,0.275032,-0.249917,0.083894,-0.391865,-0.397954,0.126516,-0.116072
8036,"[atendar, telefone, amor, deus]",2,0.622338,-0.051714,-0.179536,-0.122031,-0.425095,0.035356,-0.494136,-0.680507,...,-0.244489,0.247574,-0.175288,0.318270,-0.310399,0.197917,-0.406193,-0.360116,0.113720,-0.107785
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.627870,-0.100649,-0.166018,-0.173724,-0.449551,0.030059,-0.464931,-0.669263,...,-0.212683,0.242995,-0.158024,0.315496,-0.302949,0.177299,-0.401135,-0.378033,0.140257,-0.061338
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.637870,-0.088951,-0.163301,-0.163432,-0.460501,0.034412,-0.479991,-0.662051,...,-0.256024,0.240638,-0.175709,0.331186,-0.290355,0.194828,-0.397644,-0.399644,0.133357,-0.081277


## Separando Treino e Teste

In [35]:
target = df_word2vec['sentimentoNumerico']

In [36]:
feature = df_word2vec.iloc[:,2:102]

In [37]:
feature

,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,0.612265,-0.121134,-0.178801,-0.174028,-0.421510,0.018179,-0.478128,-0.636315,-0.093333,-0.035269,...,-0.281889,0.257685,-0.143325,0.306498,-0.234182,0.207980,-0.358231,-0.415163,0.132881,-0.113191
1,0.597896,-0.123061,-0.150154,-0.123245,-0.414788,0.018516,-0.477988,-0.622438,-0.089867,-0.028601,...,-0.258885,0.233310,-0.167665,0.288795,-0.243819,0.216081,-0.357969,-0.386116,0.109595,-0.092992
2,0.584169,-0.033485,-0.230218,-0.246397,-0.529980,-0.002390,-0.526843,-0.695480,-0.126361,0.047234,...,-0.224230,0.332945,-0.271342,0.243140,-0.256287,0.201317,-0.337849,-0.406413,0.191734,-0.127245
3,0.547829,-0.098986,-0.120857,-0.114265,-0.307566,0.005273,-0.412368,-0.573851,-0.100606,0.009933,...,-0.289230,0.198894,-0.125023,0.229357,-0.219359,0.255928,-0.285208,-0.372005,0.117037,-0.107042
4,0.614611,-0.088834,-0.184939,-0.160994,-0.445464,0.023897,-0.465764,-0.662036,-0.126526,-0.002049,...,-0.239826,0.243544,-0.161424,0.310894,-0.282802,0.195856,-0.365394,-0.403700,0.142544,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.526041,-0.066375,-0.084716,-0.112691,-0.361852,-0.064298,-0.355274,-0.652298,-0.230673,0.011531,...,-0.062480,0.128866,-0.120784,0.275032,-0.249917,0.083894,-0.391865,-0.397954,0.126516,-0.116072
8036,0.622338,-0.051714,-0.179536,-0.122031,-0.425095,0.035356,-0.494136,-0.680507,-0.160123,0.035368,...,-0.244489,0.247574,-0.175288,0.318270,-0.310399,0.197917,-0.406193,-0.360116,0.113720,-0.107785
8037,0.627870,-0.100649,-0.166018,-0.173724,-0.449551,0.030059,-0.464931,-0.669263,-0.113428,-0.015790,...,-0.212683,0.242995,-0.158024,0.315496,-0.302949,0.177299,-0.401135,-0.378033,0.140257,-0.061338
8038,0.637870,-0.088951,-0.163301,-0.163432,-0.460501,0.034412,-0.479991,-0.662051,-0.115581,-0.026380,...,-0.256024,0.240638,-0.175709,0.331186,-0.290355,0.194828,-0.397644,-0.399644,0.133357,-0.081277


In [38]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo 

In [39]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.25      0.78      0.38       360
           1       0.59      0.27      0.37       597
           2       0.42      0.13      0.20       651

    accuracy                           0.33      1608
   macro avg       0.42      0.39      0.32      1608
weighted avg       0.44      0.33      0.30      1608



In [40]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output) 

Precisão final de : 32.90%


# Rede Neural - Sequência de palavras

## Teste isolado

In [41]:
! pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pad_sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10199 sha256=35f27f0735a56d7e6c84b4aadc7911199a284629de7b0db5959dceee617a1e84
  Stored in directory: /root/.cache/pip/wheels/48/9d/22/0a6305b87a9cc46ccc032060a041c3b59f39ac462f7358997e
Successfully built pad_sequences


In [42]:
from keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences

citacao = [
    "Estudo no Inteli e escolhi o curso de Sistemas de Informação",
    "Estamos fazendo um projeto para o BTG",
    "Estamos no segundo ano de graduação",
    "Somos do grupo BTG3"
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(citacao)
citacao_token = tokenizer.texts_to_sequences(citacao)

max_length = max([len(z) for z in citacao_token]) + 1
citacao_pad = pad_sequences(citacao_token, maxlen=max_length, padding="post")

## Construção da rede neural + resultados - base tratada

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é um arquivo com a base que já passou pelo pré processamento. 

### Dataset já treinado

In [43]:
teste_rede_neural = pd.read_csv("/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada2")

In [44]:
teste_rede_neural

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


### Separação de treino e teste

In [45]:
x, y = teste_rede_neural["texto_tratado"], teste_rede_neural["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [46]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
169/169 [==============================] - ETA: 0s - loss: 1.0412 - accuracy: 0.4478

169/169 [==============================] - 4s 19ms/step - loss: 1.0412 - accuracy: 0.4478 - val_loss: 0.9822 - val_accuracy: 0.4868
Epoch 2/5
168/169 [============================>.] - ETA: 0s - loss: 0.9136 - accuracy: 0.6066

169/169 [==============================] - 4s 24ms/step - loss: 0.9134 - accuracy: 0.6066 - val_loss: 0.8682 - val_accuracy: 0.6571
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.7696 - accuracy: 0.7338

169/169 [==============================] - 6s 38ms/step - loss: 0.7696 - accuracy: 0.7338 - val_loss: 0.7937 - val_accuracy: 0.6771
Epoch 4/5
168/169 [============================>.] - ETA: 0s - loss: 0.6591 - accuracy: 0.7803

169/169 [==============================] - 5s 31ms/step - loss: 0.6592 - accuracy: 0.7804 - val_loss: 0.7495 - val_accuracy: 0.6884
Epoch 5/5
169/169 [==============================] - ETA: 0s - loss: 0.5572 - accuracy: 0.8221

83/83 [==============================] - 0s 4ms/step - loss: 0.7252 - accuracy: 0.6933
[0.7251595854759216, 0.6932931542396545]


In [47]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 4ms/step - loss: 0.7252 - accuracy: 0.6933
Loss: 0.7251595854759216
Accuracy: 0.6932931542396545


In [54]:
y_pred_probs = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1) 

classification = classification_report(y_test, y_pred_classes)

print("\nRelatório de Classificação:")
print(classification)

83/83 [==============================] - 0s 2ms/step

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.70      0.67      0.69       647
           1       0.68      0.64      0.66       956
           2       0.70      0.76      0.73      1051

    accuracy                           0.69      2654
   macro avg       0.69      0.69      0.69      2654
weighted avg       0.69      0.69      0.69      2654



### Exportação com a biblioteca pickle

In [48]:
with open('redeNeural_base.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_base.pkl', 'rb') as arquivo:
    modelo_redeNeural_base = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com cbow

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com o CBoW. 

### Dataframe do word2vec com o cbow


In [55]:
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


### Separação treino e teste

In [56]:
x, y = df_vec["Frase"], df_vec["sentimento"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [57]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
169/169 [==============================] - ETA: 0s - loss: 1.0430 - accuracy: 0.4538

169/169 [==============================] - 3s 17ms/step - loss: 1.0430 - accuracy: 0.4538 - val_loss: 0.9896 - val_accuracy: 0.4710
Epoch 2/5
169/169 [==============================] - ETA: 0s - loss: 0.9190 - accuracy: 0.6047

169/169 [==============================] - 4s 21ms/step - loss: 0.9190 - accuracy: 0.6047 - val_loss: 0.8741 - val_accuracy: 0.6424
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.7808 - accuracy: 0.7404

169/169 [==============================] - 4s 22ms/step - loss: 0.7808 - accuracy: 0.7404 - val_loss: 0.7932 - val_accuracy: 0.6718
Epoch 4/5
168/169 [============================>.] - ETA: 0s - loss: 0.6648 - accuracy: 0.7861

169/169 [==============================] - 3s 16ms/step - loss: 0.6648 - accuracy: 0.7859 - val_loss: 0.7432 - val_accuracy: 0.6858
Epoch 5/5
169/169 [==============================] - ETA: 0s - loss: 0.5656 - accuracy: 0.8190

83/83 [==============================] - 0s 2ms/step - loss: 0.7188 - accuracy: 0.6933
[0.7188287377357483, 0.6932931542396545]


In [58]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 2ms/step - loss: 0.7188 - accuracy: 0.6933
Loss: 0.7188287377357483
Accuracy: 0.6932931542396545


In [59]:
y_pred_probs = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1) 

classification = classification_report(y_test, y_pred_classes)

print("\nRelatório de Classificação:")
print(classification)

83/83 [==============================] - 0s 2ms/step

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.70      0.67      0.68       651
           1       0.72      0.62      0.67       982
           2       0.67      0.78      0.72      1021

    accuracy                           0.69      2654
   macro avg       0.70      0.69      0.69      2654
weighted avg       0.70      0.69      0.69      2654



### Exportação com a biblioteca pickle

In [60]:
with open('redeNeural_word2vec_cbow.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_cbow.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_cbow = pickle.load(arquivo)

## Construção da rede neural + resultados - word2vec com embedding layer

A rede neural abaixo foi desenvolvida com o tutorial mencionado no ínicio do notebook, que, nesse caso o dataframe utilizado é a variável do Word2Vec com Embedding Layer.

### Dataframe do word2vec com embedding layer

In [61]:
df_word2vec

,texto_tratado,sentimentoNumerico,Vetor0,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,...,Vetor90,Vetor91,Vetor92,Vetor93,Vetor94,Vetor95,Vetor96,Vetor97,Vetor98,Vetor99
0,"[alvarez, marsal, estar, conosco, sportainmet,...",1,0.612265,-0.121134,-0.178801,-0.174028,-0.421510,0.018179,-0.478128,-0.636315,...,-0.281889,0.257685,-0.143325,0.306498,-0.234182,0.207980,-0.358231,-0.415163,0.132881,-0.113191
1,"[btgpactual, with, make_repost, entendar, impa...",1,0.597896,-0.123061,-0.150154,-0.123245,-0.414788,0.018516,-0.477988,-0.622438,...,-0.258885,0.233310,-0.167665,0.288795,-0.243819,0.216081,-0.357969,-0.386116,0.109595,-0.092992
2,"[minuto, touro, ouro]",2,0.584169,-0.033485,-0.230218,-0.246397,-0.529980,-0.002390,-0.526843,-0.695480,...,-0.224230,0.332945,-0.271342,0.243140,-0.256287,0.201317,-0.337849,-0.406413,0.191734,-0.127245
3,"[querer, saber, banking, administrar]",2,0.547829,-0.098986,-0.120857,-0.114265,-0.307566,0.005273,-0.412368,-0.573851,...,-0.289230,0.198894,-0.125023,0.229357,-0.219359,0.255928,-0.285208,-0.372005,0.117037,-0.107042
4,"[início, guerra, rússia, ucrânia, velho, apare...",0,0.614611,-0.088834,-0.184939,-0.160994,-0.445464,0.023897,-0.465764,-0.662036,...,-0.239826,0.243544,-0.161424,0.310894,-0.282802,0.195856,-0.365394,-0.403700,0.142544,-0.080556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",2,0.526041,-0.066375,-0.084716,-0.112691,-0.361852,-0.064298,-0.355274,-0.652298,...,-0.062480,0.128866,-0.120784,0.275032,-0.249917,0.083894,-0.391865,-0.397954,0.126516,-0.116072
8036,"[atendar, telefone, amor, deus]",2,0.622338,-0.051714,-0.179536,-0.122031,-0.425095,0.035356,-0.494136,-0.680507,...,-0.244489,0.247574,-0.175288,0.318270,-0.310399,0.197917,-0.406193,-0.360116,0.113720,-0.107785
8037,"[grande, fiis, mercado, não, selecionar, princ...",2,0.627870,-0.100649,-0.166018,-0.173724,-0.449551,0.030059,-0.464931,-0.669263,...,-0.212683,0.242995,-0.158024,0.315496,-0.302949,0.177299,-0.401135,-0.378033,0.140257,-0.061338
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",1,0.637870,-0.088951,-0.163301,-0.163432,-0.460501,0.034412,-0.479991,-0.662051,...,-0.256024,0.240638,-0.175709,0.331186,-0.290355,0.194828,-0.397644,-0.399644,0.133357,-0.081277


### Separação treino e teste


In [62]:
x, y = df_word2vec["texto_tratado"], df_word2vec["sentimentoNumerico"]

labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

words = ["o", "ao", 'aos', 'os', 'a', 'as', 'e', 'um', 'uma', 
        'ele', 'ela', 'eles', 'elas', 'do', 'da', 'dos', 'das', 
        'de', 'no', 'na', 'nos', 'nas', 'pelo', 'pela', 'pelos', 
        'pelas', 'num', 'numa', 'nuns', 'numas', 'dum', 'duma', 
        'duns', 'dumas']

x_filter = []

for title in x:
  for word in words:
    title = title.replace(word, '')
  x_filter.append(title)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_filter)

vocab = len(tokenizer.word_docs) + 1

x_filter = tokenizer.texts_to_sequences(x_filter)

max_length = max([len(z) for z in x_filter])
x_filter = pad_sequences(x_filter, maxlen=max_length, padding='post')

x_train, x_test, y_train, y_test = train_test_split(x_filter, y, test_size=0.33)

print("Tamanho de x:", len(x_filter))
print("Tamanho de y:", len(y))


Tamanho de x: 8040
Tamanho de y: 8040


### Criação do modelo

In [63]:
model = Sequential()
model.add(Embedding(input_dim=vocab, output_dim=80, input_length=max_length, trainable = True))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(units = 3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

mc = ModelCheckpoint('weight.best.hdf5', monitor='val_acc', save_best_only=True, mode='max')

model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32, epochs = 5, callbacks = [mc])

print(model.evaluate(x_test, y_test))

Epoch 1/5
168/169 [============================>.] - ETA: 0s - loss: 1.0424 - accuracy: 0.4518

169/169 [==============================] - 5s 25ms/step - loss: 1.0422 - accuracy: 0.4519 - val_loss: 0.9854 - val_accuracy: 0.5113
Epoch 2/5
166/169 [============================>.] - ETA: 0s - loss: 0.9083 - accuracy: 0.6224

169/169 [==============================] - 3s 18ms/step - loss: 0.9075 - accuracy: 0.6222 - val_loss: 0.8750 - val_accuracy: 0.6232
Epoch 3/5
169/169 [==============================] - ETA: 0s - loss: 0.7691 - accuracy: 0.7354

169/169 [==============================] - 3s 16ms/step - loss: 0.7691 - accuracy: 0.7354 - val_loss: 0.8028 - val_accuracy: 0.6545
Epoch 4/5
169/169 [==============================] - ETA: 0s - loss: 0.6543 - accuracy: 0.7857

169/169 [==============================] - 3s 17ms/step - loss: 0.6543 - accuracy: 0.7857 - val_loss: 0.7588 - val_accuracy: 0.6692
Epoch 5/5
166/169 [============================>.] - ETA: 0s - loss: 0.5610 - accuracy: 0.8193

83/83 [==============================] - 0s 2ms/step - loss: 0.7317 - accuracy: 0.6782
[0.7316639423370361, 0.6782215237617493]


In [64]:
# Avaliação do modelo
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss:', loss)
print('Accuracy:', accuracy)

83/83 [==============================] - 0s 2ms/step - loss: 0.7317 - accuracy: 0.6782
Loss: 0.7316639423370361
Accuracy: 0.6782215237617493


In [65]:
y_pred_probs = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probs, axis=1) 

classification = classification_report(y_test, y_pred_classes)

print("\nRelatório de Classificação:")
print(classification)

83/83 [==============================] - 0s 2ms/step

Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.66      0.65      0.66       648
           1       0.67      0.64      0.65       948
           2       0.70      0.72      0.71      1058

    accuracy                           0.68      2654
   macro avg       0.67      0.67      0.67      2654
weighted avg       0.68      0.68      0.68      2654



### Exportação com a biblioteca pickle 

In [66]:
with open('redeNeural_word2vec_embedding.pkl', 'wb') as arquivo:
    pickle.dump(model, arquivo)
with open('redeNeural_word2vec_embedding.pkl', 'rb') as arquivo:
    modelo_redeNeural_word2vec_embedding = pickle.load(arquivo)